In [1]:
# 必要なパッケージを読み込む
using DelimitedFiles, DataFrames,  Plots
Plots.default(fontfamily="IPAexGothic")
# 前提として、以下のファイルが既に存在している
include("calc_IPW.jl")          # リグレット計算の基本関数
include("file_operate.jl")      # ファイル操作用関数
include("module_regret.jl") # 先ほど定義した新しいモジュール

# RankChangeIntervalモジュールを使用
using .RankChangeInterval
using Base.Threads
# 3. アベイルスペースに基づく順位変化点を分析
function analyze_rank_changes(utility_data, method_weights, t_range)
    utility_matrix = Matrix(utility_data)
    results = RankChangeInterval.analyze_all_alternatives_with_avail_space(utility_matrix, method_weights, t_range)
    
    return results
end

function count_concordant_pairs(rank1, rank2)
    n = length(rank1)

    # 各アイテムの順位（位置）を素早く引けるように辞書（Dict）を作成します。
    rank1_map = Dict(value => i for (i, value) in enumerate(rank1))
    rank2_map = Dict(value => i for (i, value) in enumerate(rank2))

    # 順位リストに含まれるアイテムを取得します。
    items = collect(keys(rank1_map))
    count = 0

    # アイテムの全てのユニークなペアについてループします。
    for i in 1:n
        for j in (i+1):n
            item_a = items[i]
            item_b = items[j]
            is_a_before_b_in_1 = rank1_map[item_a] < rank1_map[item_b]
            is_a_before_b_in_2 = rank2_map[item_a] < rank2_map[item_b]

            # 両方のリストで順序関係が同じであれば、カウントを増やします。
            if is_a_before_b_in_1 == is_a_before_b_in_2
                count += 1
            end
        end
    end

    return count
end

   Resolving package versions...
  No Changes to `/workspaces/inulab_julia_devcontainer/Project.toml`
  No Changes to `/workspaces/inulab_julia_devcontainer/Manifest.toml`


count_concordant_pairs (generic function with 1 method)

In [2]:
const repeat_num = 100
const criteria_num = 6
const utility_matrix_num = 100
const weight_type = ["A", "B", "C", "D", "E"]
method_name_list = [    "/EV","/GM","/WMIN","/DMIN",
"/MMRW","/MMRD","/E-MMRW","/G-MMRW","/E-MMRD","/G-MMRD",
"/MMRwc","/eMMRw","/eMMRwc","/gMMRw","/gMMRwc",
"/eMMRd", "/eMMRdc", "/gMMRd", "/gMMRdc",
"/AMRW","/AMRD","/E-AMRW","/G-AMRW","/E-AMRD","/G-AMRD",
"/AMRwc","/eAMRw","/eAMRwc","/gAMRw","/gAMRwc",
"/eAMRd", "/eAMRdc", "/gAMRd", "/gAMRdc", 
"/ReAMRwc","/ReMMRwc","/RgAMRwc","/RgMMRwc"]
const utility_methods = ["u1", "u2"]

# --- 問題があった組み合わせを格納するための配列 ---
problematic_combinations = []

# --- メイン処理 ---
for utility in utility_methods
    for type in weight_type
        @threads for method in method_name_list
            tid = threadid()
            # println("$utility;$type:$method を処理中, スレッドID: ", tid)
            base_path = "/workspaces/inulab_julia_devcontainer/data/a3/regret/$utility/N=6"
            dir_path = joinpath(base_path, type * method)

            # 1. ディレクトリの存在をチェック
            if !isdir(dir_path)
                push!(problematic_combinations, (utility, type, method, "ディレクトリが見つかりません"))
                continue # 次のメソッドへ
            end

            file_path = joinpath(dir_path, "$(utility)_minimax_regret_$(repeat_num).csv")

            # 2. CSVファイルの存在をチェック
            if !isfile(file_path)
                push!(problematic_combinations, (utility, type, method, "CSVファイルが見つかりません"))
                continue # 次のメソッドへ
            end

            # 3. CSVファイルの内容をチェック
            try
                lines = readlines(file_path)

                if isempty(lines)
                    push!(problematic_combinations, (utility, type, method, "CSVファイルが空です"))
                    continue
                end

                # 後ろから最大50行（またはファイルの全行）を取得
                lines_to_check = lines[max(1, end-49):end]

                found_target_line = false
                for line in lines_to_check
                    parts = split(line, ',')
                    if length(parts) >= 2 &&
                       parts[1] == string(utility_matrix_num) &&
                       parts[2] == string(repeat_num)
                        found_target_line = true
                        break # 条件に合う行が見つかったのでループを抜ける
                    end
                end

                # 末尾50行を調べても条件に合う行がなかった場合
                if !found_target_line
                    push!(problematic_combinations, (utility, type, method, "条件を満たす行がCSVの末尾50行にありません"))
                end

            catch e
                # ファイル読み込み中にエラーが発生した場合
                push!(problematic_combinations, (utility, type, method, "ファイル読み込みエラー: $e"))
            end
        end
    end
end

# --- 最終結果の出力 ---
println("==================================================")
if isempty(problematic_combinations)
    println("✅ すべての組み合わせをチェックし、問題は見つかりませんでした。")
else
    println("⚠️ 以下の組み合わせで問題が検出されました：$(length(problematic_combinations))件")
    for (ut, ty, me, reason) in problematic_combinations
        println("Utility: $ut, Type: $ty, Method: $me-> 理由: $reason")
    end
end
println("==================================================")

⚠️ 以下の組み合わせで問題が検出されました：91件
Utility: u1, Type: A, Method: /eMMRwc-> 理由: 条件を満たす行がCSVの末尾50行にありません
Utility: u1, Type: A, Method: /gAMRd-> 理由: 条件を満たす行がCSVの末尾50行にありません
Utility: u1, Type: A, Method: /gAMRw-> 理由: 条件を満たす行がCSVの末尾50行にありません
Utility: u1, Type: A, Method: /gAMRdc-> 理由: CSVファイルが見つかりません
Utility: u1, Type: A, Method: /gAMRwc-> 理由: CSVファイルが見つかりません
Utility: u1, Type: A, Method: /eMMRd-> 理由: 条件を満たす行がCSVの末尾50行にありません
Utility: u1, Type: A, Method: /eMMRdc-> 理由: CSVファイルが見つかりません
Utility: u1, Type: A, Method: /eAMRdc-> 理由: 条件を満たす行がCSVの末尾50行にありません
Utility: u1, Type: A, Method: /G-MMRW-> 理由: 条件を満たす行がCSVの末尾50行にありません
Utility: u1, Type: B, Method: /gAMRw-> 理由: CSVファイルが見つかりません
Utility: u1, Type: B, Method: /gAMRd-> 理由: CSVファイルが見つかりません
Utility: u1, Type: B, Method: /eMMRd-> 理由: CSVファイルが見つかりません
Utility: u1, Type: B, Method: /gAMRwc-> 理由: CSVファイルが見つかりません
Utility: u1, Type: B, Method: /gAMRdc-> 理由: CSVファイルが見つかりません
Utility: u1, Type: B, Method: /eMMRdc-> 理由: CSVファイルが見つかりません
Utility: u1, Type: B, Method: /

In [ ]:
# =================================================================
# ▼▼▼ 修正後のコード ▼▼▼
# =================================================================

# --- 最初に処理すべき(type, method)の組み合わせリストを作成 ---
const task_list = [(t, m) for t in weight_type for m in method_name_list]

const problematic_set = Set([(item[1], item[2], item[3]) for item in problematic_combinations])

# --- メインループ ---
for utility in utility_methods
    utility_data = read_utility_value(utility)

    # typeとmethodの組み合わせで並列化
    @threads for (type, method) in task_list
        # 現在の組み合わせが「問題のあったリスト」に含まれているかチェック
        if !((utility, type, method) in problematic_set)
            # 含まれていない場合（=正常な組み合わせ）は、このループをスキップ
            continue
        end

        # `type`に依存する処理は、並列ループの中に移動させる
        trueW = read_true_weights(type)
        s_range_true = find_optimal_trange(trueW.L, trueW.R)

        tid = threadid()
        println("$utility;$type:$method を処理中, スレッドID: ", tid)

        filepath = "/workspaces/inulab_julia_devcontainer/data/a3/regret/$utility/N=6/$type$method"
        if !isdir(filepath)
            mkdir(filepath)
        end
        filepath = filepath * "/$(utility)_minimax_regret_$repeat_num.csv"

        methodW = read_method_weights(type * method, repeat_num, criteria_num)

        open(filepath, "w") do io
            for utl_num in 1:utility_matrix_num
                true_result = analyze_rank_changes(utility_data[utl_num], trueW, s_range_true)
                for Repeat_time in 1:repeat_num
                    t_range = find_optimal_trange(methodW[Repeat_time].L, methodW[Repeat_time].R)
                    method_results = analyze_rank_changes(utility_data[utl_num], methodW[Repeat_time], t_range)
                    
                    true_rank_change_time = length(true_result.change_points)
                    method_rank_change_time = length(method_results.change_points)
                    
                    count_pair_concodance = Array{Int}(undef, method_rank_change_time, true_rank_change_time)
                    for true_t in 1:true_rank_change_time
                        for method_t in 1:method_rank_change_time
                            count_pair_concodance[method_t, true_t] = count_concordant_pairs(
                                method_results.rankings[method_results.change_points[method_t]],
                                true_result.rankings[true_result.change_points[true_t]]
                            )
                        end
                    end
                    
                    println(io, join([utl_num, Repeat_time, true_rank_change_time, method_rank_change_time], ','))
                    println(io, join(["", true_result.change_points...], ','))
                    for i in 1:method_rank_change_time
                        method_t = method_results.change_points[i]
                        concordance_row = count_pair_concodance[i, :]
                        println(io, join([method_t, concordance_row...], ','))
                    end
                end
            end
        end
    end
end

u1;C:/eMMRwc を処理中, スレッドID: 10
u1;D:/eAMRdc を処理中, スレッドID: 1
u1;D:/eMMRwc を処理中, スレッドID: 7
u1;B:/gAMRw を処理中, スレッドID: 13
u1;A:/eMMRwc を処理中, スレッドID: 12
u1;A:/G-MMRW を処理中, スレッドID: 2
u1;E:/gAMRw を処理中, スレッドID: 8
u1;A:/gAMRw を処理中, スレッドID: 9
u1;C:/gAMRd を処理中, スレッドID: 5
u1;C:/gAMRw を処理中, スレッドID: 16
u1;E:/eMMRdc を処理中, スレッドID: 3
u1;B:/eMMRwc を処理中, スレッドID: 14
u1;D:/gMMRdc を処理中, スレッドID: 6
u1;E:/eMMRwc を処理中, スレッドID: 4
